In [ ]:
import imaplib
import email
from email.header import decode_header
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool
from rich.console import Console
from rich.markdown import Markdown
from langchain.agents import create_agent

# --- CONFIGURATION ---
EMAIL_USER = ""
EMAIL_PASS = ""
GEMINI_API_KEY = ""

# --- 1. CORE EMAIL LOGIC ---
def fetch_latest_emails_logic(count):
    """Internal helper to talk to Gmail."""
    try:
        mail = imaplib.IMAP4_SSL("imap.gmail.com")
        mail.login(EMAIL_USER, EMAIL_PASS)
        mail.select("inbox")

        _, messages = mail.search(None, "ALL")
        email_ids = messages[0].split()[-count:]
        
        all_emails_text = ""
        for e_id in reversed(email_ids):
            _, data = mail.fetch(e_id, "(RFC822)")
            msg = email.message_from_bytes(data[0][1])
            
            # Decode Subject & Sender
            subject, encoding = decode_header(msg["Subject"])[0]
            if isinstance(subject, bytes):
                subject = subject.decode(encoding or "utf-8")
            
            sender = msg.get("From")
            
            body = ""
            if msg.is_multipart():
                for part in msg.walk():
                    if part.get_content_type() == "text/plain":
                        body = part.get_payload(decode=True).decode(errors='ignore')
                        break
            else:
                body = msg.get_payload(decode=True).decode(errors='ignore')
                
            all_emails_text += f"\n---\nFROM: {sender}\nSUBJECT: {subject}\nBODY: {body[:1500]}\n"
        
        mail.logout()
        return all_emails_text
    except Exception as e:
        return f"Error connecting to email: {str(e)}"

# --- 2. DEFINE AGENT TOOLS ---
@tool
def fetch_emails(count: int = 5):
    """Use this tool to get the latest emails from the user's inbox."""
    return fetch_latest_emails_logic(count)

@tool
def search_sender(sender_name: str):
    """Use this tool to find emails from a specific person or company name."""
    # (Simplified for now, just fetches 10 and agent filters them)
    return fetch_latest_emails_logic(10)

# --- 3. SET UP THE AGENT BRAIN ---
llm = ChatGoogleGenerativeAI(
    google_api_key=GEMINI_API_KEY, 
    model="gemini-flash-latest", 
    temperature=0
)

tools = [fetch_emails, search_sender]

# This is the "Instruction Manual" for the agent
system_instruction = """
You are an Elite Executive Assistant. Your job is to process the user's inbox 
and provide a highly structured, actionable briefing.

### BRIEIFING STRUCTURE:
## 📬 Inbox Intelligence Brief
---
### 🔴 URGENT / ACTION REQUIRED
* **Sender**: [Summary of action and 📅 deadline if found]

### 🟡 FYI / UPDATES
* **Sender**: [Brief 1-sentence summary]

### 🟢 NO ACTION / COMPLETED
* **Sender**: [Category: News, Newsletter, or Receipt]
"""


# Build the Agent & Executor
agent = create_agent(llm, tools=tools)


# --- 4. EXECUTION & DISPLAY ---
if __name__ == "__main__":
    console = Console()
    
    console.print("[bold cyan]🤖 Agent is checking your mail...[/bold cyan]")
    
    # The agent decides WHICH tool to call based on this natural language request
    query = "Summarize my last 5 emails and highlight anything important or with a deadline."
    query2 = "Summarize any email from Daisy Goswami."
    result = agent.invoke({"messages":[{"role": "system", "content": system_instruction}, {"role": "user", "content": query}]})
    print("\n" + "="*30)
    print("YOUR DAILY EMAIL BRIEF")
    print("="*30)
    console = Console()
    # This will render the bold text, emojis, and bullets in color!
    # 1. Get the raw response
    # 2. Extract content safely
    if isinstance(result["messages"][-1].content, list):
        # If it's a list, join the text parts together
        text_content = "".join([part["text"] for part in result["messages"][-1].content if "text" in part])
    else:
        # If it's already a string, just use it
        text_content = result["messages"][-1].content

    # 3. Render with Rich
    print("\n") # Add some breathing room
    md = Markdown(text_content)
    console.print(md)
    # print(result["messages"][-1].content)